In [1]:
# CREATE TABLE items (id bigserial PRIMARY KEY, content TEXT, embedding vector(1536));

In [2]:
import os
from dotenv import find_dotenv, load_dotenv
from openai import OpenAI
import openai
import psycopg2

In [3]:
load_dotenv(find_dotenv())

True

In [4]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
# Set up your OpenAI API key
# openai.api_key = OPENAI_API_KEY
client = OpenAI()
# Choose a model
model = "text-embedding-ada-002"
print(OPENAI_API_KEY[:12])

sk-proj-pqjN


In [ ]:
print("Connecting to PostgreSQL...")
conn = psycopg2.connect(
    database="postgres",
    user="postgres",
    password="postgres",
    host="host.docker.internal",
)

cur = conn.cursor()

print("Successfully connected to PostgreSQL.")

Connecting to PostgreSQL...
Successfully connected to PostgreSQL.


In [6]:
# Converting the prompt to the pgvector embedding
def get_embedding(prompt):
    response = openai.embeddings.create(input=prompt, model="text-embedding-ada-002")

    embedding = response.data[0].embedding

    # Converting the embedding to the pgvector and returning it
    return "[" + ",".join(map(str, embedding)) + "]"

Second, find the most relevant movies for a provided user prompt by calculating the cosine distance (`<=>`) between the prompt's and movies' embeddings:


In [7]:
try:
    sentences = [
        "For weather use this tool => **get_weather**.",
        "For hotel booking use this tool => **get_hotel_booking**.",
        "For car hire use this tool => **get_car_hire**.",
        "For flight booking use this tool => **get_flight**.",
        "For jokes use this tool => **get_joke**.",
        "To add two numbers use this tool => **get_adder**.",
        "To critique an article prior to publication but not after publication use this tool => **get_article_review**.",
        "For jokes use this tool => **get_joke**.",
        "For all other requests use this tool => **customer_service_agent**.",
        "For weather use this tool => **get_weather**.",
        "For hotel booking use this tool => **get_hotel_booking**.",
        "For car hire use this tool => **get_car_hire**.",
        "For flight booking use this tool => **get_flight**.",
        "For jokes use this tool => **get_joke**.",
        "To add two numbers use this tool => **get_adder**.",
        "To critique an article prior to publication but not after publication use this tool => **get_article_review**.",
        "For jokes use this tool => **get_joke**.",
        "For all other requests use this tool => **customer_service_agent**.",
        "We only sell shoes.",
        "Our opening hours are Monday to Friday from 9am to 5pm.",
        "We are located at 123 Main Street, Brighton",
        "We specialise in red shoes but have all colours",
        "Our VAT rate is 20 percent and is applicable on all sales",
        "We only accept card payments",
    ]
    TABLE_NAME = "items"
    # Insert sentences into the items table
    for sentence in sentences:
        embedding = get_embedding(sentence)
        cur.execute(
            f"INSERT INTO {TABLE_NAME}  (content, embedding) VALUES (%s, %s)",
            (sentence, embedding),
        )
        conn.commit()
        count = cur.rowcount
        print(
            count,
            f"Record inserted successfully into {TABLE_NAME} table",
        )
except Exception as e:
    print("Error executing query", str(e))

1 Record inserted successfully into items table
1 Record inserted successfully into items table
1 Record inserted successfully into items table
1 Record inserted successfully into items table
1 Record inserted successfully into items table
1 Record inserted successfully into items table
1 Record inserted successfully into items table
1 Record inserted successfully into items table
1 Record inserted successfully into items table
1 Record inserted successfully into items table
1 Record inserted successfully into items table
1 Record inserted successfully into items table
1 Record inserted successfully into items table
1 Record inserted successfully into items table
1 Record inserted successfully into items table
1 Record inserted successfully into items table
1 Record inserted successfully into items table
1 Record inserted successfully into items table
1 Record inserted successfully into items table
1 Record inserted successfully into items table
1 Record inserted successfully into item

In [9]:
try:
    # Example query
    # query = "Which report for sales"
    query = "Tell me jokes"
    query = "Which report for sales"
    # query = "Sunset"

    query_embedding = get_embedding(query)

    # Perform a cosine similarity search
    cur.execute(
        """
        SELECT id, content, 1 - (embedding <=> %s) AS cosine_similarity
            FROM items
            ORDER BY cosine_similarity DESC LIMIT 3
        """,
        (query_embedding,),
    )

    # Fetch and print the result
    print("Query:", query)
    print("Most similar sentences:")
    for row in cur.fetchall():
        print(f"ID: {row[0]},  Cos: {row[2]}, CONTENT: {row[1]},")

except Exception as e:
    print("Error executing query", str(e))



Query: Which report for sales
Most similar sentences:
ID: 21,  Cos: 0.8841870772281201, CONTENT: SALES REPORT: We use this to calculate sales for any period,
ID: 22,  Cos: 0.8345963114667602, CONTENT: P and L Report: When we want to know the profit for the company we use this report,
ID: 68,  Cos: 0.7852703928947504, CONTENT: Our VAT rate is 20 percent and is applicable on all sales,


In [ ]:
cur.close()
conn.close()